# 4. 단어 추출 word extraction

## 4-a. 단어 추출을 하는 이유
- 커뮤니티 text는 신조어, 은어가 많다.
- 토큰화를 하기 위해선, 신조어와 은어를 추출해야 한다.

## 4-b. 단어 추출 계획
1. text로만 이루어진 txt 파일 만들기
2. soynlp 의 WordExtractor 를 이용하여 단어 추출
3. 단어와 수치 데이터를 csv 파일로 저장

## 4-c. 단어 추출 진행

### 4-c-1. 파일 불러오기

In [1]:
import pandas as pd
df = pd.read_csv("3_spaced_text.csv", encoding="utf-8")
df.head(3)

,search_keyword,date_created,time_created,writer,is_reply,id,spaced_text
0,JYP,2023-05-16,17:52:40,ㅇㅇ(118.34),0,10100730087,야 개보지년들아 JYP 주가 봤냐.. 미쳣다 지금... 개잡주 오르는건 그러려니 하...
1,JYP,2023-05-16,17:52:48,ㅇㅇ(118.34),1,10100730087,ㅠㅠ
2,JYP,2023-05-16,18:04:04,ㅇㅇ(203.227),1,10100730087,ㅠㅠ


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891014 entries, 0 to 891013
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   search_keyword  891014 non-null  object
 1   date_created    891014 non-null  object
 2   time_created    891014 non-null  object
 3   writer          891014 non-null  object
 4   is_reply        891014 non-null  int64 
 5   id              891014 non-null  int64 
 6   spaced_text     891014 non-null  object
dtypes: int64(2), object(5)
memory usage: 47.6+ MB


### 4-c-2. corpus를 list 형태로 만든다

In [3]:
corpus = df['spaced_text'].tolist()

### 4-c-3. word_extractor 학습시키기

In [4]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(corpus) # list of str or like
words = word_extractor.extract()

training was done. used memory 2.503 Gbse memory 3.045 Gb
all cohesion probabilities was computed. # words = 27749
all branching entropies was computed # words = 1146192
all accessor variety was computed # words = 1146192


In [5]:
words['주식']

Scores(cohesion_forward=0.415731390877544, cohesion_backward=0.1350615196769043, left_branching_entropy=4.853872802910785, right_branching_entropy=4.579661565493153, left_accessor_variety=577, right_accessor_variety=561, leftside_frequency=39894, rightside_frequency=2876)

In [6]:
len(words)

14096

### 4-c-4. word extraction 결과 출력

In [10]:
# 결과 출력

import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy, word_score)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[1000:2000]:
    print('%s     (%d, %.3f, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy,
            word_score(score)
            )
         )

단어   (빈도수, cohesion, branching entropy, word_score)

마이크론도     (161, 0.275, 3.827, 12.639)
깨지면     (296, 0.294, 3.761, 12.633)
이제     (11713, 0.064, 5.282, 12.621)
성공하면     (262, 0.261, 3.879, 12.609)
개미가     (1075, 0.134, 4.547, 12.602)
냠냠     (260, 0.884, 2.656, 12.589)
백신     (1491, 0.280, 3.805, 12.584)
카카오게임즈는     (154, 0.366, 3.537, 12.580)
삼성은     (816, 0.116, 4.684, 12.580)
뇌피셜     (385, 0.501, 3.222, 12.568)
높은     (2523, 0.278, 3.811, 12.565)
턴어라운드     (155, 0.816, 2.734, 12.562)
점유율이     (226, 0.337, 3.616, 12.542)
과천은     (368, 0.188, 4.198, 12.541)
장기적으로는     (127, 0.360, 3.551, 12.525)
산업이     (354, 0.210, 4.089, 12.516)
받으면     (473, 0.193, 4.170, 12.514)
있을까?     (582, 0.184, 4.218, 12.503)
설마     (1135, 0.149, 4.432, 12.495)
나는     (4625, 0.060, 5.331, 12.479)
회장이     (477, 0.141, 4.479, 12.470)
IT     (1236, 0.334, 3.620, 12.464)
페이스북     (317, 0.466, 3.286, 12.460)
있음?     (1520, 0.128, 4.581, 12.458)
아닌가?     (594, 0.165, 4.324, 12.448)
잡아서     (261, 0.174, 4.269, 1

### 4-c-5. threshold 판단
- 눈으로 직접 살펴보며, 무의미한 결과를 거를 word_score의 threshold를 정한다
- 살펴본 결과. word_score >= 7.117 인 값만 유의미한 데이터로 간주한다
- 결과 dict : scored_words

In [11]:
threshold = 7.117
# threshold 를 기준으로 dictionary 생성
scored_words = {word: word_score(score) for word, score in words.items() if word_score(score) >= threshold}

print(scored_words)

{'승인': 13.302589630653603, '넣고': 10.274301407819458, '허허': 17.31489473934168, '내일': 23.22667123179534, '질문': 7.72881258165684, '될듯': 16.283147243650404, '없고': 10.770713349282584, '난다': 12.160814038190463, '동안': 14.839553444266304, '함?': 14.93147566083983, '섹터': 31.534060436720786, '적자': 8.45803276558843, '받고': 7.399890049628643, '콘솔': 8.16577504631527, '평단': 9.656159487219293, '넥슨': 21.556479755430825, '엔솔': 31.659481418164727, '놓고': 57.89811105959774, '라고': 20.7153499134393, 'yg': 9.179338677759697, '그럼': 9.766213714516656, '올해': 14.44465572377283, '업체': 12.323817315146384, '폭락': 9.142573595379133, '본다': 9.842543952193605, '환율': 9.850259681852743, '봄.': 23.542169840435193, '국내': 15.80101642788103, '좋은': 13.35042629194444, '수익': 8.690323968258811, '홀딩': 11.562922036163792, '오늘': 43.65462283652787, '어제': 14.84106246193826, '//': 38.611438505324216, '잡주': 17.793951220136698, '짜리': 24.46741010941641, '쓰는': 10.083837277155396, '산업': 25.932241417816734, '함께': 19.365294752327525, '빼고': 19.99

In [120]:
filter_은는 = ['엔솔은', '걔들은', '미국은', '시장은', '내일은', '삼성은', '과천은', '국장은', '니들은', '넥슨은', '지금은', '주식은', '오늘은', '애플은', '기업은', '요즘은', '한국은', '종목은', '산업은', '사람은', '놈들은', '하닉은', '매출은', '애들은', '좆같은', '삼전은', '게임은', '일본은', '실적은', '금양은', '쿠팡은', '결론은', '가격은', '괜찮은', '똑같은', '새끼들은', '사람들은', '조선장은', '당분간은', '병신같은', '종목들은', '기업들은', '넷마블은', '점유율은', '개미들은', '주주들은', '코스닥은',
 '연기금은', '회사들은', 'LG화학은', '영업이익은', '검은사막은', '마이크론은', '셀트리온은', 'lg화학은', '이새끼들은', '에코프로비엠은']
filter_로다에 = ['프로', '호로', '으로', '앞으로', '주식으로', '생각보다', '모르겠다', '떨어진다', '감사합니다', '에코프로보다',
                 '코갤에', '고점에', '만원에', '에코에', '시장에', '때문에', '주식에', '국장에', '이번에', '하루에',  
                 '최근에', '엘앤에', '작년에', '아침에', '나중에', '만원대에', '에코프로에', '에코호로에']
filter_이 = ['놈이', '것이', '네이', '억이',  '중이', '말이', '신이',  '답이', '장이', '돈이', '시총이', '수익이', 
               '세력이', '실적이', '포홀이', '애들이', '사람이', '삼전이', '신성이', '비엠이', '조정이', '국장이', 
               '병신이', '만원이', '가격이', '시장이', '개인이', '고점이', '지금이', '매출이', '거품이', '생각이',
               '기업이', '놈들이', '기관이', '종목이', '니들이', '오늘이', '영업이', '수급이', '주식이', '금양이',
               '상승이', '느낌이', '가능성이', '개미들이', '미친듯이', '코스닥이', '사람들이', '새끼들이', '에코프로비엠이']
filter_가까지부터처럼을를 = ['드가', '미래가', '새끼가', '자체가', '회사가', '에코가', '개미가', '주가가', '모두가', '이유가', '테슬라가',
                    '차전지가', '초전도체가', '에코프로가', '에코호로가', '까지', '만까지', '년까지', 
                    '부터', '월부터', '처럼', '에코프로처럼', '돈을', '주식을', '에코프로를']
filter_etc = ['에코프', '에코프로로', 'ㅋㅋㅋ코프로', '엔비', '에코프로비', '엔비디', '테슬', '초전', '차전', '갑자',
              '에코프로도', '에코프로의', '에코프로만', '에코프로랑', '에코프로나', '에코도', 'ㅋㅋㅋ캬ㅋㅋㅋ', '공매', '에코랑',
              '병신들', '병신아', '병신들아', '병신새끼', '개병신', '이새끼들', '저새끼', '이새끼', '셀트리', '곱버', 
              '익절하', '궁금하', '감사합니', '레인보', '생각하', '세력들', '새끼들', '병신들', '틀딱들', '기관들', '사람들', 
              '형제들', '주주들', '주식들', '개미들', '종목들', '아줌마들', '이새끼들']

## 4-e. 단어 리스트를 파일로 저장

### 4-e-1. 파일로 저장하기

In [127]:
# 파일로 저장
filename = "4_ecopro_word_list.txt"
words = scored_words_3
with open(filename, "w", encoding="utf-8") as file:
    for word in words:
        file.write(word + "\n")

print("파일 저장 완료.")

파일 저장 완료.


### 4-e-2. 불러와서 확인하기

In [128]:
# 파일로부터 문자열 리스트 불러오기
with open(filename, "r", encoding="utf-8") as file:
    word_list = [line.rstrip('\n') for line in file]

print("불러온 문자열 리스트:")
print(word_list)

불러온 문자열 리스트:
['현재', '없어', '예정', '결정', '바로', '않음', '이거', '입갤', '잔고', '버핏', '중에', 'ㅅㄱ', '반등', '지수', '하면', '모든', '수익', '서울', '살까', '당장', '해도', '운지', '형님', '조정', '할거', '흐름', '진입', '층에', '흑우', '경쟁', '제가', '증가', '갈때', '제일', '졸업', '인데', '높음', '맨날', '매수', '장기', '털고', '잡고', '거의', '병신', '좋아', '구조', '기관', '길게', '피자', '성장', '결국', '박살', '뉴스', '별로', '샀어', '특징', '대한', '않나', '맞냐', '소재', '난리', '찍고', '본다', '몇개', '랠리', '양극', '믿는', '심리', '갔다', '똑같', '방어', '회복', '규모', '밸류', '빠진', '맞음', '매매', '바닥', '되냐', '수준', '올해', '이런', '방법', '너네', '축하', '누구', '무슨', '관심', '해서', '벌써', '섹터', '엘앤', '큐ㅠ', '급락', '언제', '했다', '에코', '테마', '추매', '정리', '억원', '부자', '쉽게', '그냥', '달러', '방금', '없다', '하락', '봐야', '승리', '되나', '맞다', 'ㅆㅂ', '너무', '확률', '직원', '소리', '싸게', '벌고', '연상', '많음', '재미', '허허', '물린', '분명', '은행', 'ㅅㅂ', '머리', '캬ㅋ', '되지', '슈카', '다음', '걱정', '적정', '형들', '앞으', '구간', '살껄', '상승', '넘게', '삼성', '포홀', '국내', '영익', '그런', '산업', '보고', '미래', '남들', '굳이', '오를', '분기', '크게', 'ㅎㅎ', '팔면', '진짜', '확정', '두산', '선동', '유증', '전부', '백만', '버스', '저점', '호